## Using Gradio to create a simple interface.

Check out the library on [github](https://github.com/gradio-app/gradio-UI) and see the [getting started](https://gradio.app/getting_started.html) page for more demos.

Now we'll wrap this function with a Gradio interface.

> ### <a> Code for this [Space]()

In [ ]:
!pip install argparse
!pip install gradio
!pip install transformers[sentencepiece]

In [ ]:
from transformers import AutoTokenizer, SwitchTransformersForConditionalGeneration, AutoModelForSequenceClassification
import torch
import gradio as gr
import argparse
from scipy.special import softmax
import csv
import urllib.request
import numpy as np
import requests

In [ ]:
args_dict = dict( 
    EX_LIST = [["This is wonderful!"],
                ["Nice car"],
                ["La France est la meilleure équipe du monde"],
                ["Visca Barca"],
                ["Hala Madrid"],
                ["Buongiorno"],
                # ["Auf einigen deutschen Straßen gibt es kein Radar"],
                ["Tempo soleggiato in Italia"],
                ["Bonjour"],
                ["صباح الخير"],
                ["اكل زوجتي جميل"],
               ],

    #MMiniLM
    # Load the pretrained model and tokenizer
    tokenizer_MMiniLM = AutoTokenizer.from_pretrained("Karim-Gamal/MMiniLM-L12-finetuned-emojis-IID-Fed"),
    model_MMiniLM = AutoModelForSequenceClassification.from_pretrained("Karim-Gamal/MMiniLM-L12-finetuned-emojis-IID-Fed"),

    #XLM
    # Load the pretrained model and tokenizer
    tokenizer_XLM = AutoTokenizer.from_pretrained("Karim-Gamal/XLM-Roberta-finetuned-emojis-IID-Fed"),
    model_XLM = AutoModelForSequenceClassification.from_pretrained("Karim-Gamal/XLM-Roberta-finetuned-emojis-IID-Fed"),

    #Bert
    # Load the pretrained model and tokenizer
    tokenizer_Bert = AutoTokenizer.from_pretrained("Karim-Gamal/BERT-base-finetuned-emojis-IID-Fed"),
    model_Bert = AutoModelForSequenceClassification.from_pretrained("Karim-Gamal/BERT-base-finetuned-emojis-IID-Fed"),
    


    description = 'Real-time Emoji Prediction',
    article = '<head><style>@import url(https://fonts.googleapis.com/css?family=Open+Sans:400italic,600italic,700italic,800italic,400,600,700,800)<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3" crossorigin="anonymous"> <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.7.2/font/bootstrap-icons.css"> <link rel="stylesheet" href="https://unpkg.com/bootstrap-table@1.21.2/dist/bootstrap-table.min.css">\
    .table-responsive{-sm|-md|-lg|-xl} body{ background-color: #f5f5f5; padding: 120px 0; font-family: \'Open Sans\', sans-serif; } img{ max-width:100%; } .div_table_{ position:relative; width: max-content; margin:0 auto; } .profile-card{ position:relative; width:280px; margin:0 auto; padding:40px 30px 30px; background:#fff; border: 5px solid rgba(255,255,255,.7); text-align:center; border-radius:40px; transition: all 200ms ease; } .profile-card_2{ position:relative; width:60%; // margin:0 auto; padding:40px 30px 30px; background:#fff; border: 5px solid rgba(255,255,255,.7); text-align:center; border-radius:40px; transition: all 200ms ease; } .mask-shadow{ z-index:-1 !important; width:95%; height:12px; background:#000; bottom:0; left:0; right:0; margin:0 auto; position:absolute; border-radius:4px; opacity:0; transition: all 400ms ease-in; } .mask-shadow_2{ z-index:-1 !important; width:95%; height:12px; background:#000; bottom:0; left:0; right:0; margin:0 auto; position:absolute; border-radius:4px; opacity:0; transition: all 400ms ease-in; } .profile-card:hover{ box-shadow: 0px 30px 60px -5px rgba(55,55,71,0.3); transform: translate3d(0,-5px,0); .mask-shadow{ opacity:1; box-shadow: 0px 30px 60px -5px rgba(55,55,71,0.3); position:absolute; } } .profile-card_2:hover{ box-shadow: 0px 30px 60px -5px rgba(55,55,71,0.3); transform: translate3d(0,-5px,0); .mask-shadow{ opacity:1; box-shadow: 0px 30px 60px -5px rgba(55,55,71,0.3); position:absolute; } } .profile-card header{ display:block; margin-bottom:10px; } .profile-card_2 header{ display:block; margin-bottom:10px; } .profile-card header a{ width:150px; height:150px; display:block; border-radius:100%; margin:-120px auto 0; box-shadow: 0 0 0 5px #82b541; } .profile-card_2 header a{ width:85%; height:85%; display:block; border-radius:10%; margin:-120px auto 0; box-shadow: 0 0 0 5px #82b541; } .profile-card header a img{ border-radius: 50%; width:150px; height:150px; } .profile-card_2 header a img{ border-radius: 10%; width:100%; height:100%; } .profile-card:hover header a, .profile-card header a:hover{ animation: bounceOut .4s linear; -webkit-animation: bounceOut .4s linear; } .profile-card_2:hover header a, .profile-card header a:hover{ animation: bounceOut .4s linear; -webkit-animation: bounceOut .4s linear; } .profile-card header h1{ font-size:20px; padding:20px; color:#444; text-transform:uppercase; margin-bottom:5px; } .profile-card_2 header h1{ font-size:20px; padding:20px; color:#444; text-transform:uppercase; margin-bottom:5px; } .profile-card header h2{ font-size:14px; color:#acacac; text-transform:uppercase; margin:0; } .profile-card_2 header h2{ font-size:14px; color:#acacac; text-transform:uppercase; margin:0; } /*content*/ .profile-bio{ font-size:14px; color:#a5a5a5; line-height:1.7; font-style: italic; margin-bottom:30px; } /*link social*/ .profile-social-links{ margin:0; padding:0; list-style:none; } .profile-social-links li{ display: inline-block; margin: 0 10px; } .profile-social-links li a{ width: 55px; height:55px; display:block; background:#f1f1f1; border-radius:50%; -webkit-transition: all 2.75s cubic-bezier(0,.83,.17,1); -moz-transition: all 2.75s cubic-bezier(0,.83,.17,1); -o-transition: all 2.75s cubic-bezier(0,.83,.17,1); transition: all 2.75s cubic-bezier(0,.83,.17,1); transform-style: preserve-3d; } .profile-social-links li a img{ width:35px; height:35px; margin:10px auto 0; } .profile-social-links li a:hover{ background:#ddd; transform: scale(1.2); -webkit-transform: scale(1.2); } /*animation hover effect*/ @-webkit-keyframes bounceOut { 0% { box-shadow: 0 0 0 4px #82b541; opacity: 1; } 25% { box-shadow: 0 0 0 1px #82b541; opacity: 1; } 50% { box-shadow: 0 0 0 7px #82b541; opacity: 1; } 75% { box-shadow: 0 0 0 4px #82b541; opacity: 1; } 100% { box-shadow: 0 0 0 5px #82b541; opacity: 1; } } @keyframes bounceOut { 0% { box-shadow: 0 0 0 6px #82b541; opacity: 1; } 25% { box-shadow: 0 0 0 2px #82b541; opacity: 1; } 50% { box-shadow: 0 0 0 9px #82b541; opacity: 1; } 75% { box-shadow: 0 0 0 3px #82b541; opacity: 1; } 100% { box-shadow: 0 0 0 5px #82b541; opacity: 1; } }</style></head>',
    

    )

config = argparse.Namespace(**args_dict)

In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    text = text.lower()
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
        # print(" ".join(new_text))
    return " ".join(new_text)

In [ ]:
def test_with_sentance(text ,net ,tokenizer):

  # text = "good morning"
  text = preprocess(text)

  # tc = TweetCleaner(remove_stop_words=True, remove_retweets=False)
  # print('before : ' ,text)
  # text = tc.get_cleaned_text(text)
  # print('after : ' ,text)

  net.eval()
  encoded_input = tokenizer.encode(text, padding=True, truncation=True, return_tensors='pt')
  net.to('cpu')
  # print(type())
  # encoded_input = {k: v.to(DEVICE) for k, v in encoded_input.items()}
  output = net(encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  # download label mapping
  labels=[]
  mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emoji/mapping.txt"
  with urllib.request.urlopen(mapping_link) as f:
      html = f.read().decode('utf-8').split("\n")
      csvreader = csv.reader(html, delimiter='\t')
  labels = [row[1] for row in csvreader if len(row) > 1]

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  output_d = {}
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      # print(f"{ranking[i]}) {l} {np.round(float(s), 4)}")
      output_d[l] = np.round(float(s), 4)

      if i == 2 :
        # break
        return output_d

  # net.to('cuda:0')


In [ ]:
list_interface = []
list_title = []

> ## <a> MMiniLM

In [ ]:
def _method(text):

  # tokenizer = AutoTokenizer.from_pretrained(config.CHECKPOINT_BERT)
  # model_loaded = torch.load('/content/NEW_MODELS_Imbalance/Bert/g_ex3_bert_multi_fed_data_epoch_2.pt', map_location=torch.device('cpu'))

  return test_with_sentance(text , config.model_MMiniLM , config.tokenizer_MMiniLM)

# greet("sun")

interface = gr.Interface(
    
    fn = _method, 
    
    inputs=gr.Textbox(placeholder="Enter sentence here..."), 
    outputs="label",
    examples=config.EX_LIST,
    live = True,
    
    
    title = 'MiniLM Multilingual',
    
    description=config.description,
    article = '',
    
)
list_interface.append(interface)
list_title.append('MiniLM Multilingual')

> ## <a> XLM

In [ ]:
def _method(text):

  # tokenizer = AutoTokenizer.from_pretrained(config.CHECKPOINT_BERT)
  # model_loaded = torch.load('/content/NEW_MODELS_Imbalance/Bert/g_ex3_bert_multi_fed_data_epoch_2.pt', map_location=torch.device('cpu'))

  return test_with_sentance(text , config.model_XLM , config.tokenizer_XLM)

# greet("sun")

interface = gr.Interface(
    
    fn = _method, 
    
    inputs=gr.Textbox(placeholder="Enter sentence here..."), 
    outputs="label",
    examples=config.EX_LIST,
    live = True,
    
    
    title = 'XLM Roberta Multilingual',
    
    description=config.description,
    article = '',
    
)
list_interface.append(interface)
list_title.append('XLM Roberta Multilingual')

> ## <a> BERT

In [ ]:
def _method(text):

  # tokenizer = AutoTokenizer.from_pretrained(config.CHECKPOINT_BERT)
  # model_loaded = torch.load('/content/NEW_MODELS_Imbalance/Bert/g_ex3_bert_multi_fed_data_epoch_2.pt', map_location=torch.device('cpu'))

  return test_with_sentance(text , config.model_Bert , config.tokenizer_Bert)

# greet("sun")

interface = gr.Interface(
    
    fn = _method, 
    
    inputs=gr.Textbox(placeholder="Enter sentence here..."), 
    outputs="label",
    examples=config.EX_LIST,
    live = True,
    
    
    title = 'BERT Multilingual',
    
    description=config.description,
    article = '',
    
)
list_interface.append(interface)
list_title.append('BERT Multilingual')

> ## <a> Switch

In [ ]:
API_URL_Switch = "https://api-inference.huggingface.co/models/Karim-Gamal/switch-base-8-finetuned-SemEval-2018-emojis-IID-Fed"
headers_Switch = {"Authorization": "Bearer hf_"}


def query_Switch(payload):
    response = requests.post(API_URL_Switch, headers=headers_Switch, json=payload)
    return response.json()

query_Switch({	"inputs": 'test',})


[{'generated_text': 'joy joy joy joy joy joy joy joy joy joy joy joy joy joy joy joy joy joy joy'}]

In [ ]:
def _method(text):
  text = preprocess(text)
  output_temp = query_Switch({
  "inputs": text,
  })

  text_to_emoji = {'red' : '❤', 'face': '😍', 'joy':'😂', 'love':'💕', 'fire':'🔥', 'smile':'😊', 'sunglasses':'😎', 'sparkle':'✨', 'blue':'💙', 'kiss':'😘', 'camera':'📷', 'USA':'🇺🇸', 'sun':'☀' , 'purple':'💜', 'blink':'😉', 'hundred':'💯', 'beam':'😁', 'tree':'🎄', 'flash':'📸', 'tongue':'😜'}

  # Extract the dictionary from the list
  d = output_temp[0]

  # Extract the text from the 'generated_text' key
  text = d['generated_text']

  # my_dict = {}
  # my_dict[str(text_to_emoji[text.split(' ')[0]])] = 0.99
  return text_to_emoji[text.split(' ')[0]]
  

# greet("sun")

interface = gr.Interface(
    
    fn = _method, 
    
    inputs=gr.Textbox(placeholder="Enter sentence here..."), 
    outputs="text",
    examples=config.EX_LIST,
    live = True,
    
    
    title = 'Switch-Base-8',
    
    description=config.description,
    article = '',
    
)
list_interface.append(interface)
list_title.append('Switch-Base-8')

> ## About us

In [ ]:
def _method(input_rating):

  # tokenizer = AutoTokenizer.from_pretrained(config.CHECKPOINT_BERT)
  # model_loaded = torch.load('/content/NEW_MODELS_Imbalance/Bert/g_ex3_bert_multi_fed_data_epoch_2.pt', map_location=torch.device('cpu'))

  if input_rating <=2:
    return {'🔥': 0.6, '✨': 0.3, '💯': 0.1}

  elif input_rating <= 4 and input_rating >2:
    return {'✨': 0.6, '😉': 0.3, '💯': 0.1}

  elif input_rating >4:
    return {'😍': 0.6, '💯': 0.3, '💕': 0.1}

  # return test_with_sentance(text , config.model_loaded_bert_multi_NONIID , config.tokenizer_bert)

# greet("sun")

interface = gr.Interface(
    
    fn = _method, 
    
    inputs=gr.Slider(1, 5, value=4),
    outputs="label",
    # examples=config.EX_LIST,
    live = True,
    
    
    title = 'About us',
    
    description='We don\'t have sad emoji so our rating will always be great. 😂',
    
    # CSS Source : https://codepen.io/bibiangel199/pen/warevP

    article = config.article + '<!-- this is the markup. you can change the details (your own name, your own avatar etc.) but don’t change the basic structure! --> <div class="div_table_"> <table class="table"> <tr> <td><aside class="profile-card"> <div class="mask-shadow"></div> <header> <!-- here’s the avatar --> <a href="https://www.linkedin.com/in/hossam-amer-23b9329b/"> <img src="https://drive.google.com/uc?export=view&id=1-C_UIimeqbofJC_lldC7IQzIOX_OYRSn"> </a> <!-- the username --> <h1 style = " font-size:20px; padding:20px; color:#444;  margin-bottom:5px; " >Dr. Hossam Amer</h1> <!-- and role or location --> <h2 style = "  font-size:14px; color:#acacac; text- margin:0; " >Research Scientist at Microsoft</h2> </header> </aside></td> </tr> </table> </div> <div class="div_table_"> <table class="table"> <tr> <td><aside class="profile-card"> <div class="mask-shadow"></div> <header> <!-- here’s the avatar --> <a href="https://www.linkedin.com/in/ahmed-mohamed-gaber-143b25175/"> <img src="https://drive.google.com/uc?export=view&id=1OiGZwhL23PYhIJzQexYvPDFRrgUIprMj"> </a> <!-- the username --> <h1 style = " font-size:20px; padding:20px; color:#444;  margin-bottom:5px; ">Ahmed Gaber</h1> <!-- and role or location --> <h2 style = "  font-size:14px; color:#acacac; text- margin:0; " >Master\'s student at Queen\'s University</h2> </header> </aside></td> <td><aside class="profile-card"> <div class="mask-shadow"></div> <header> <!-- here’s the avatar --> <a href="https://www.linkedin.com/in/karim-gamal-mahmoud/"> <img src="https://drive.google.com/uc?export=view&id=1Lg2RzimITL9y__X2hycBTX10rJ4o87Ax"> </a> <!-- the username --> <h1 style=" font-size:20px; padding:20px; color:#444;  margin-bottom:5px; ">Karim Gamal</h1> <!-- and role or location --> <h2 style = "  font-size:14px; color:#acacac; text- margin:0; " >Master\'s student at Queen\'s University</h2> </header> </aside></td> </tr> </table> </div>',
    )


list_interface.append(interface)
list_title.append('About us')

In [ ]:
demo = gr.TabbedInterface(
    list_interface, 
    list_title,
    title='Multilingual Emoji Prediction Using Federated Learning',
    css='.gradio-container {color : orange}',)
    # css='.gradio-container {background-color: white; color : orange}',)
demo.launch()

/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:716: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:716: UserWarning: api_name predict already exists, using predict_2
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:716: UserWarning: api_name predict already exists, using predict_3
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:716: UserWarning: api_name predict already exists, using predict_4
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>